<h2>1. Importar librerías</h2>

In [9]:
import cv2
import os
from PIL import Image
import pandas as pd
import numpy as np

<h2>2. Definir las carpetas y variables</h2>

In [7]:
# Configuración
carpeta_origen = "./catchas_renombrados" 
carpeta_segmentos = "segmentos" 
sep = [8, 34, 60, 86, 112, 138, 164]
carpeta_binarios = "imagenes_binarias"

<h2>3. Segmentamos las imagenes</h2>

<h3>3.1. Segmentamos verticalmente</h3>

In [5]:
# Crear las carpetas de destino para los segmentos
carpetas_destino = ["1", "2", "3", "4", "5", "6"]
for carpeta in carpetas_destino:
    os.makedirs(os.path.join(carpeta_segmentos, carpeta), exist_ok=True)

# Renombrar archivos y que no se sobrescriban
def obtener_nombre_unico(carpeta_destino, nombre_segmento):
    # Generar la ruta completa del archivo
    ruta_completa = os.path.join(carpeta_destino, nombre_segmento)
    contador = 1
    
    # Si el archivo ya existe, agregar un número entre paréntesis
    base, ext = os.path.splitext(nombre_segmento)
    while os.path.exists(ruta_completa):
        # Crear un nuevo nombre con el contador
        nombre_segmento = f"{base}({contador}){ext}"
        ruta_completa = os.path.join(carpeta_destino, nombre_segmento)
        contador += 1

    return nombre_segmento

# Procesar cada imagen en la carpeta origen
for nombre_imagen in os.listdir(carpeta_origen):
    # Ruta completa de la imagen
    ruta_imagen = os.path.join(carpeta_origen, nombre_imagen)

    # Verificar si es un archivo de imagen
    if not (ruta_imagen.endswith(".jpg") or ruta_imagen.endswith(".png")):
        continue

    # Leer la imagen
    img = cv2.imread(ruta_imagen)
    if img is None:
        print(f"No se pudo leer la imagen: {nombre_imagen}")
        continue

    # Verificar que el nombre tenga al menos 6 caracteres
    base_nombre = os.path.splitext(nombre_imagen)[0]  # Quitar extensión
    if len(base_nombre) < 6:
        print(f"Nombre de archivo demasiado corto: {nombre_imagen}")
        continue

    # Cortar las 6 partes centrales (excluye las extremas)
    for i in range(len(sep) - 1):  # Ignorar el primer y último segmento
        y1, y2 = sep[i], sep[i + 1]  # Coordenadas de corte
        segmento = img[:, y1:y2]     # Segmentar verticalmente (todas las filas, columnas entre y1 y y2)

        # Usar el carácter correspondiente del nombre de la imagen
        nombre_segmento = f"{base_nombre[i]}.png"  # i para alinearse con las 6 letras de la imagen
        carpeta_segmento = carpetas_destino[i]  # Carpetas en el orden de "1", "2", "3", etc.

        # Obtener un nombre único para evitar sobrescrituras
        nombre_segmento_unico = obtener_nombre_unico(os.path.join(carpeta_segmentos, carpeta_segmento), nombre_segmento)

        # Ruta completa para guardar el archivo
        ruta_guardado = os.path.join(carpeta_segmentos, carpeta_segmento, nombre_segmento_unico)

        # Guardar el segmento
        cv2.imwrite(ruta_guardado, segmento)

    os.remove(ruta_imagen)

print("Segmentación completada.")

Segmentación completada.


<h3>3.2. Recortamos los bordes por arriba y abajo</h3>

In [6]:
def readImg(path):
    images = []
    nombres = []
    for img_name in os.listdir(path):
        filename = path + img_name

        # lee la imagen en gris
        G = cv2.imread(filename,0)
        
        #agrega la imagen a una lista y los nombres de la ruta de donde salieron
        images.append(G)
        nombres.append(filename)

    return images, nombres

def recortar_h(carpeta, y1, y2):
    lista_img, list_nom = readImg(carpeta)
    i = 0
    camb = 0
    n_camb = 0

    for img in lista_img:
        if(len(img)>(y2-y1)):
            new_img = img[y1:y2, :] 
            cv2.imwrite(list_nom[i], new_img)
            camb += 1
        else:
            n_camb += 1
        i += 1
    print(f"Se cambiaron {camb} archivos de tamaño y {n_camb} no se alteraron, de un total de {camb+n_camb} archivos en la carpeta {carpeta}")
    return

#segmentar las imagenes en horizontal
recortar_h(f'./{carpeta_segmentos}/1/', 10, 40)
recortar_h(f'./{carpeta_segmentos}/3/', 10, 40)
recortar_h(f'./{carpeta_segmentos}/5/', 10, 40)

recortar_h(f'./{carpeta_segmentos}/2/', 20, 50)
recortar_h(f'./{carpeta_segmentos}/4/', 20, 50)
recortar_h(f'./{carpeta_segmentos}/6/', 20, 50)

Se cambiaron 3500 archivos de tamaño y 0 no se alteraron, de un total de 3500 archivos en la carpeta ./segmentos/1/
Se cambiaron 3500 archivos de tamaño y 0 no se alteraron, de un total de 3500 archivos en la carpeta ./segmentos/3/
Se cambiaron 3500 archivos de tamaño y 0 no se alteraron, de un total de 3500 archivos en la carpeta ./segmentos/5/
Se cambiaron 3500 archivos de tamaño y 0 no se alteraron, de un total de 3500 archivos en la carpeta ./segmentos/2/
Se cambiaron 3500 archivos de tamaño y 0 no se alteraron, de un total de 3500 archivos en la carpeta ./segmentos/4/
Se cambiaron 3500 archivos de tamaño y 0 no se alteraron, de un total de 3500 archivos en la carpeta ./segmentos/6/


<h2>4. Convertir a binarios los segmentos y juntarlos en una carpeta</h2>

In [8]:
os.makedirs(os.path.join(carpeta_binarios), exist_ok=True)

def obtener_nombre_unico(carpeta_destino, nombre_segmento):
    # Generar la ruta completa del archivo
    ruta_completa = os.path.join(carpeta_destino, nombre_segmento)
    contador = 1
    
    # Si el archivo ya existe, agregar un número entre paréntesis
    base, ext = os.path.splitext(nombre_segmento)
    while os.path.exists(ruta_completa):
        # Crear un nuevo nombre con el contador
        nombre_segmento = f"{base}({contador}){ext}"
        ruta_completa = os.path.join(carpeta_destino, nombre_segmento)
        contador += 1

    return nombre_segmento

def convertir_bin(carpeta_origen):
    cont = 0
    for nombre_imagen in os.listdir(carpeta_origen): 
        cont += 1

        ruta_imagen = os.path.join(carpeta_origen, nombre_imagen)

        # Abrimos la imagen
        imagen = Image.open(ruta_imagen)

        # Convertimos a escala de grises
        imagen_gris = imagen.convert('L')

        # Definimos un umbral, por prueba y error nos quedamos con 128
        umbral = 128
        imagen_binaria = imagen_gris.point(lambda p: p > umbral and 255)

        # Quitar extensión para guardar el carácter del inicio
        nombre_base = f"{os.path.splitext(nombre_imagen)[0][0]}.png"
        nombre_unico = obtener_nombre_unico(carpeta_binarios, nombre_base)
        ruta_guardado = os.path.join(carpeta_binarios, nombre_unico)
        # Guarda la imagen binaria
        imagen_binaria.save(ruta_guardado)
        # Borramos la imagen original
        os.remove(ruta_imagen)

    print(f"Se convirtieron {cont} imagenes a imagenes binarias de la carpeta {carpeta_origen}")
    return


convertir_bin(f'./{carpeta_segmentos}/1/')
convertir_bin(f'./{carpeta_segmentos}/2/')
convertir_bin(f'./{carpeta_segmentos}/3/')
convertir_bin(f'./{carpeta_segmentos}/4/')
convertir_bin(f'./{carpeta_segmentos}/5/')
convertir_bin(f'./{carpeta_segmentos}/6/')

Se convirtieron 3500 imagenes a imagenes binarias de la carpeta ./segmentos/1/
Se convirtieron 3500 imagenes a imagenes binarias de la carpeta ./segmentos/2/
Se convirtieron 3500 imagenes a imagenes binarias de la carpeta ./segmentos/3/
Se convirtieron 3500 imagenes a imagenes binarias de la carpeta ./segmentos/4/
Se convirtieron 3500 imagenes a imagenes binarias de la carpeta ./segmentos/5/
Se convirtieron 3500 imagenes a imagenes binarias de la carpeta ./segmentos/6/


<h2>5. Crear el dataset</h2>

In [10]:
vectores = []
categorias = []

for nombre_imagen in os.listdir(carpeta_binarios):
    ruta_imagen = os.path.join(carpeta_binarios, nombre_imagen)

    # Abrimos la imagen y sacamos su matriz
    imagen = Image.open(ruta_imagen)
    matriz = np.array(imagen)

    # Transformamos la imagen en un vector 1D
    vectorized_image = matriz.flatten()

    vectores.append(vectorized_image)
    categorias.append(str(os.path.splitext(nombre_imagen)[0][0]))

# Crear el DataFrame
df = pd.DataFrame(vectores, columns=[f'Pixel{i+1}' for i in range(780)])
df['Categoria'] = categorias  # Añadir la columna de categorías

print(df.head())

df.to_csv('dataset.csv', index=False)

   Pixel1  Pixel2  Pixel3  Pixel4  Pixel5  Pixel6  Pixel7  Pixel8  Pixel9  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   Pixel10  ...  Pixel772  Pixel773  Pixel774  Pixel775  Pixel776  Pixel777  \
0        0  ...         0         0         0         0         0         0   
1        0  ...         0         0         0         0         0         0   
2        0  ...         0         0         0         0         0         0   
3        0  ...         0         0         0         0         0         0   
4        0  ...         0         0         0         0         0         0   

   Pixel778  Pixel779  Pixel780  Categoria  
0         0      

<h2>6. Limpieza de datos del dataset</h2>

In [11]:
# Cargar el dataset proporcionado para analizar su contenido
file_path = 'dataset.csv'
df = pd.read_csv(file_path)

# Limpiando los datos
data_clean = df.drop_duplicates()

# Normalizar valores de píxeles entre 0 y 1 (Para ayudar a los modelos)
pixel_columns = data_clean.columns[:-1]  # Excluir 'Categoria'
data_clean[pixel_columns] = data_clean[pixel_columns] // 255

print(data_clean)

# Guardamos el dataset limpio
data_clean.to_csv('dataset_clean.csv', index=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_18844\3343526694.py:3: DtypeWarning: Columns (780) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
C:\Users\USER\AppData\Local\Temp\ipykernel_18844\3343526694.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean[pixel_columns] = data_clean[pixel_columns] // 255


       Pixel1  Pixel2  Pixel3  Pixel4  Pixel5  Pixel6  Pixel7  Pixel8  Pixel9  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
5           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
20991       0       0       0       0       0       0       0       0       0   
20992       0       0       0       0       0       0       0       0       0   
20995       0       0       0       0       0       0       0       0       0   
20997       0       0       0       0       0       0       0       0       0   
20999       0       0       0       0       0       0       0       0       0   

       Pixel10  ...  Pixel7